In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/train_6BJx641.csv
/kaggle/input/test_pavJagI.csv


In [2]:
from sklearn import tree, ensemble, metrics, linear_model, preprocessing, model_selection, feature_selection
from plotnine import *
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Create class which performs Label Encoding - if required
class categorical_encoder:
    def __init__(self, columns, kind = 'label', fill = True):
        self.kind = kind
        self.columns = columns
        self.fill = fill
        
    def fit(self, X):
        self.dict = {}
        self.fill_value = {}
        
        for col in self.columns:
            label = preprocessing.LabelEncoder().fit(X[col])
            self.dict[col] = label
            
            # To fill
            if self.fill:
                self.fill_value[col] = X[col].mode()[0]
                X[col] = X[col].fillna(self.fill_value[col])
                
        print('Label Encoding Done for {} columns'.format(len(self.columns)))
        return self
    def transform(self, X):
        for col in self.columns:
            if self.fill:
                X[col] = X[col].fillna(self.fill_value[col])
                
            X.loc[:, col] = self.dict[col].transform(X[col])
        print('Transformation Done')
        return X
    
def categorical(df):
    df['var2'] = df['var2'].replace({'A': 1, 'B': 2, 'C': 3})
    return df



In [4]:
test = pd.read_csv('/kaggle/input/test_pavJagI.csv')
train = pd.read_csv('/kaggle/input/train_6BJx641.csv')

train['datetime'] = pd.to_datetime(train['datetime'], yearfirst = True)
test['datetime'] = pd.to_datetime(test['datetime'], yearfirst = True)

In [5]:
# The following approach of merging train and test works for competitions but not during production because in production 
# it is required to replicate the operations performed on the training set on the test set. However, if done correctly, the 
# results will be similar in the approaches.

# Indicators
train['which'] = 1
test['which'] = 0

# Merge
data = pd.concat([train, test], axis = 0, ignore_index = True)
data = data.set_index('datetime')
data = data.sort_index()

############  Create New Features - with different lags ################
data['temperature_rolling'] = data['temperature'].rolling('24H').mean()
data['var1_rolling'] = data['var1'].rolling('24H').mean()
data['windspeed_rolling'] = data['windspeed'].rolling('24H').mean()

# Week lagged features
data['temperature_rolling_week'] = data['temperature'].rolling('168H').mean()
data['var1_rolling_week'] = data['var1'].rolling('168H').mean()
data['windspeed_rolling_week'] = data['windspeed'].rolling('168H').mean()

# Other Lags
data['temperature_rolling_3d'] = data['temperature'].rolling('72H').mean()
data['temperature_rolling_6h'] = data['temperature'].rolling('6H').mean()
data = data.reset_index()


# y lagged values
temp_med = data.set_index('datetime').resample('2M')['electricity_consumption'].mean()
temp_med.name = 'y_med_month'
data = pd.concat([data.set_index('datetime'),temp_med], 
          axis = 1).fillna(method = 'bfill').reset_index()

# Split Back
train = data.loc[data['which'] == 1, :].drop('which', axis = 1)
test = data.loc[data['which'] == 0, :].drop(['which', 'electricity_consumption'], axis = 1)

In [6]:
# Create Time Series datetime features
def ts_features(df, col = 'datetime'):
    #df['dayofmonth'] = df[col].dt.day
    df['weekday'] = df[col].dt.dayofweek
    df['weekend'] = (df[col].dt.dayofweek >= 5)*1
    df['month'] = df[col].dt.month
    df['hour'] = df[col].dt.hour
    df['year'] = df[col].dt.year
    return df

In [7]:
X_train = ts_features(train).drop(['datetime', 'electricity_consumption', 'ID'], axis = 1)
y_train = ts_features(train)['electricity_consumption']

X_test = ts_features(test).drop(['ID', 'datetime'], axis = 1)

test_id = test.ID
train_id = train.ID

X_train = categorical(X_train)
X_test = categorical(X_test)

In [8]:
X_train.head()

,temperature,var1,pressure,windspeed,var2,temperature_rolling,var1_rolling,windspeed_rolling,temperature_rolling_week,var1_rolling_week,windspeed_rolling_week,temperature_rolling_3d,temperature_rolling_6h,y_med_month,weekday,weekend,month,hour,year
0,-11.4,-17.1,1003.0,571.910,1,-11.400000,-17.100,571.910000,-11.400000,-17.100,571.910000,-11.400000,-11.400000,240.217391,0,0,7,0,2013
1,-12.1,-19.3,996.0,575.040,1,-11.750000,-18.200,573.475000,-11.750000,-18.200,573.475000,-11.750000,-11.750000,240.217391,0,0,7,1,2013
2,-12.9,-20.0,1000.0,578.435,1,-12.133333,-18.800,575.128333,-12.133333,-18.800,575.128333,-12.133333,-12.133333,240.217391,0,0,7,2,2013
3,-11.4,-17.1,995.0,582.580,1,-11.950000,-18.375,576.991250,-11.950000,-18.375,576.991250,-11.950000,-11.950000,240.217391,0,0,7,3,2013
4,-11.4,-19.3,1005.0,586.600,1,-11.840000,-18.560,578.913000,-11.840000,-18.560,578.913000,-11.840000,-11.840000,240.217391,0,0,7,4,2013


In [9]:
X_test = X_test.fillna(method = 'ffill')

## 2 Approaches
1. Single Model(LightGBM)
2. Ensemble Model(LightGBM+XGBoost+CatBoost+RandomForest)

**Single Model - LightGBM Approach**

In [10]:
import lightgbm as lgb
model = lgb.LGBMRegressor(n_estimators = 4000, learning_rate = .02, 
                          max_features = .7, max_depth = 3, subsample = .9).fit(X_train, y_train)


In [11]:
# Make Submission
submission = pd.DataFrame()
submission['ID'] = test_id

# Take a weighted average of the predictions
submission['electricity_consumption'] = model.predict(X_test)

# Save submission file
submission.to_csv('/kaggle/working/submission.csv', index = None)

**Ensemble Model Approach**      

Here, the idea is to train several models and to take a weighted average of their predictions. I have chosen the weights by trial and error. 

In [12]:
from sklearn import pipeline
import xgboost as xgb
import catboost as cb

model = ensemble.RandomForestRegressor(n_estimators = 250, max_depth = 20, min_samples_leaf = 5,
                                       n_jobs = 4).fit(X_train, y_train)
model1 = lgb.LGBMRegressor(n_estimators = 4000, learning_rate = .02, max_features = .7, max_depth = 3, subsample = .9).fit(X_train, y_train)
model2 = xgb.XGBRegressor(n_estimators = 2000, learning_rate = .04, max_features = .7, max_depth = 3, subsample = .9).fit(X_train, y_train)
model4 = cb.CatBoostRegressor(n_estimators = 2000, learning_rate = .04, max_depth = 3,
                              rsm = .7, subsample = .9, silent = True).fit(X_train, y_train)

In [13]:
submission = pd.DataFrame()
submission['ID'] = test_id

# Take a weighted average of the predictions
submission['electricity_consumption'] = (model.predict(X_test)+(3*model1.predict(X_test)+(2*model2.predict(X_test))+\
                                                               (3*model4.predict(X_test))))/9
submission.to_csv('/kaggle/working/submission.csv', index = None)